In [16]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("preprocessed_apartment_rentals_no_state_dummies_nocluster.csv")

# Features und Ziel
X = data.drop(columns=["price", "cityname"])
y = data["price"]

# Numerische Spalten
num_cols = X.select_dtypes(include=["int64", "float64"]).columns

# Skalieren
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])





# Regression

# Ridge Regression

In [17]:
from Nested_CV_targetEncoding import NestedCVRegressorWithTargetEncoding

In [18]:
from sklearn.linear_model import Ridge
import numpy as np
ridge_params = {"alpha": np.logspace(-3, 3, 10)}

ridge_cv = NestedCVRegressorWithTargetEncoding(model = Ridge(), param_grid = ridge_params, encode_cols=["state"])
ridge_cv.run(X, y, ouput=True)



TypeError: NestedCVRegressorWithTargetEncoding.run() got an unexpected keyword argument 'ouput'

In [ ]:
number_outer_cv_splits = 5
model_evaluation = pd.DataFrame({"Fold": range(1, number_outer_cv_splits+1)})
model_evaluation['Modell'] = ['Ridge'] * number_outer_cv_splits
model_evaluation['R_2'] = ridge_cv.get_r2_scores()
model_evaluation['MSE'] = ridge_cv.get_mse_scores()
model_evaluation["runtime"] = ridge_cv.get_fit_times()
model_evaluation

AttributeError: 'NestedCVRegressorWithTargetEncoding' object has no attribute 'get_r2_scores'

In [8]:
model_evaluation.to_pickle('benchmarking/Ridgeregression_targetencoding.pkl')

# Analyse
Welche Preise werden gut, welche schlechter vorhergesagt?

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 1️⃣ Best Alpha aus Nested CV
best_parameters = ridge_cv.get_best_params()
alpha_values = [float(d['alpha']) for d in best_parameters]
mean_alpha = np.mean(alpha_values)
print("Final Alpha:", mean_alpha)


# 2️⃣ Daten splitten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
plt.hist(y_test)
plt.show()
# 3️⃣ Finales Ridge-Modell trainieren
final_model = Ridge(alpha=mean_alpha)
final_model.fit(X_train, y_train)

# 4️⃣ Vorhersagen auf Testdaten
y_pred = final_model.predict(X_test)

# 5️⃣ Scores berechnen
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Test MSE:", mse)
print("Test R²:", r2)

# 6️⃣ Residual Plot: echte Preise vs absolute Fehler
abs_error = y_test - y_pred

plt.figure(figsize=(8,5))
plt.scatter(y_test, abs_error, c= X_test[:,1], alpha=0.5)
scatter = plt.scatter(y_test, abs_error, c=X_test[:,5], alpha=0.5, cmap='viridis')
plt.xlabel("True Prices (y_test)")
plt.ylabel("Absolute Error |y_true - y_pred|")
plt.title("Residual Plot: True Prices vs Prediction Error")
plt.grid(True)
plt.legend()
cbar = plt.colorbar(scatter)
cbar.set_label('Feature X_test[:,5]')
plt.show()



KeyError: 'alpha'